In [35]:
import numpy as np
import matplotlib.pyplot as plt
import random
from numba import njit
from sklearn.cluster import KMeans
from scipy import stats

N=1024
var_J=1

@njit
def initial_lattice(N):
    M=np.ones((N))
    l=np.array([1,-1])
    for i in range(N):
            M[i]=np.random.choice(l)
    return(M)

@njit
def intraction(N,mean_J,var_J):
    J_matrix=np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i<j :
                J_matrix[i][j]=random.gauss(mean_J/N,var_J/np.sqrt(N))
            else :
                J_matrix[i][j]=0
    J_matrix=J_matrix+np.transpose(J_matrix)  
    return(J_matrix)

@njit
def energy_local(M,J_matrix,i):
    H=np.sum(-J_matrix[i]*M[i]*M)
    return(H)

@njit
def energy_total(M,J_matrix,N):
    E_t=0
    for i in range(N):
        E_t=E_t+energy_local(M,J_matrix,i)
    E_t=E_t/2
    return(E_t)


@njit
def trial(J_matrix,N,T): 
    field_list=np.zeros(N)
    M=initial_lattice(N)
    E_t=energy_total(M,J_matrix,N)
    for trial in range(int(np.sqrt(N**3))):
        random_i=random.randrange(N)                
        M[random_i]=M[random_i]*(-1)
        delta_E=2*energy_local(M,J_matrix,random_i)
        if delta_E>0:
            p=np.exp(-delta_E/T)
            coin=random.random()
            if coin>p :
                M[random_i]=M[random_i]*(-1)
            else :
                E_t=E_t+delta_E
        else :
            E_t=E_t+delta_E
    
    for i in range(N):
        field=np.sum(-J_matrix[i]*M)
        field_list[i]=field

    return(field_list)

#spin glass
mean_feature_SG=[]
var_feature_SG=[]
for mean_J in np.arange(0.000001,0.010001,0.001):
    for T in np.arange(0.000001,0.010001,0.001):
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        mean_feature_SG.append(abs(np.mean(field_time_series)))
        var_feature_SG.append(np.var(field_time_series))

#paramagnetic  
mean_feature_P=[]
var_feature_P=[]
for mean_J in np.arange(0.000001,0.010001,0.001):
    for T in np.arange(1000,1000.01,0.001):
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        mean_feature_P.append(abs(np.mean(field_time_series)))
        var_feature_P.append(np.var(field_time_series))

#ferro    
mean_feature_F=[]
var_feature_F=[]
for mean_J in np.arange(1000,1000.01,0.001):
    for T in np.arange(0.000001,0.010001,0.001):
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        mean_feature_F.append(abs(np.mean(field_time_series))/mean_J)
        var_feature_F.append(np.var(field_time_series))

In [36]:
# Shapiro-Wilk test
shapiro_test_mean_feature_SG = stats.shapiro(mean_feature_SG)
shapiro_test_var_feature_SG = stats.shapiro(var_feature_SG)
shapiro_test_mean_feature_P = stats.shapiro(mean_feature_P)
shapiro_test_var_feature_P = stats.shapiro(var_feature_P)
shapiro_test_mean_feature_F = stats.shapiro(mean_feature_F)
shapiro_test_var_feature_F = stats.shapiro(var_feature_F)
print("SW_test mean_SG:",shapiro_test_mean_feature_SG)
print("SW_test mean_SG:",shapiro_test_var_feature_SG)
print("SW_test mean_P:",shapiro_test_mean_feature_P)
print("SW_test mean_P:",shapiro_test_var_feature_P)
print("SW_test mean_F:",shapiro_test_mean_feature_F)
print("SW_test mean_F:",shapiro_test_var_feature_F)

SW_test mean_SG: ShapiroResult(statistic=0.9193630814552307, pvalue=1.3023184692428913e-05)
SW_test mean_SG: ShapiroResult(statistic=0.9907974004745483, pvalue=0.7288612127304077)
SW_test mean_P: ShapiroResult(statistic=0.9196752309799194, pvalue=1.3527736882679164e-05)
SW_test mean_P: ShapiroResult(statistic=0.9833914637565613, pvalue=0.24183231592178345)
SW_test mean_F: ShapiroResult(statistic=0.9897857308387756, pvalue=0.647283673286438)
SW_test mean_F: ShapiroResult(statistic=0.9858376383781433, pvalue=0.36430859565734863)
